In [46]:
import pandas as pd
from pandas import *
import numpy as np
from datetime import datetime, timedelta
import regex as re
import os
import seaborn as sns
from openpyxl import load_workbook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [47]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/SALT/Dashboards/')

In [48]:
df = pd.read_csv("./CBAScoredFiles/dp_assessment_296.csv")

/var/folders/0c/r0hw428532s7bttp9my0ym_00000gn/T/ipykernel_50761/3692428526.py:1: DtypeWarning: Columns (6,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./CBAScoredFiles/dp_assessment_296.csv")


In [49]:
main = [
 'disecode',
 'student_name',
 'class',
 'unique_id',
 'gender',
 'social_category',
'EVS_present_absent',
'Maths_present_absent',
'Language_present_absent'] 

In [50]:
score_list = [i for i in df if i.startswith('Score_')]

In [51]:
df_unpivot = pd.melt(df,id_vars = main
                        ,value_vars = score_list)

In [52]:
df_unpivot[['S','Q']] = df_unpivot['variable'].str.split('_',1,expand=True)

In [53]:
df_unpivot[['Subject','Question']] = df_unpivot['Q'].str.split('Q',1,expand=True)

In [54]:
df_unpivot.loc[df_unpivot["Subject"]=="E","Subject"]="EVS"
df_unpivot.loc[df_unpivot["Subject"]=="L","Subject"]="Telugu"
df_unpivot.loc[df_unpivot["Subject"]=="M","Subject"]="Maths"

In [55]:
df_unpivot = df_unpivot.drop(['variable','S','Q'],axis=1)

In [56]:
df_unpivot.fillna('',inplace = True)

In [57]:
indexELM = df_unpivot[(
                          ((df_unpivot['EVS_present_absent']=='A')&(df_unpivot['Subject']=='EVS'))|
                          ((df_unpivot['Language_present_absent']=='A')&(df_unpivot['Subject']=='Telugu'))|
                          ((df_unpivot['Maths_present_absent']=='A')&(df_unpivot['Subject']=='Maths'))|
                          ((df_unpivot['EVS_present_absent']=='')&(df_unpivot['Subject']=='EVS'))|
                          ((df_unpivot['Language_present_absent']=='')&(df_unpivot['Subject']=='Telugu'))|
                          ((df_unpivot['Maths_present_absent']=='')&(df_unpivot['Subject']=='Maths'))
                        )
                         ].index
df_unpivot.drop(indexELM, inplace=True)

In [58]:
df_unpivot = df_unpivot.drop(['EVS_present_absent','Language_present_absent','Maths_present_absent'],axis=1)

In [59]:
df_unpivot = df_unpivot.rename(columns = {'disecode':"Udise", 'student_name':'Child Name','class':'Grade','unique_id':'Child Id','gender':'Gender','social_category':'Social_Category','Question':'Question no','value':'Question Score'})


In [60]:
school_mapping = pd.read_excel('./MappingFiles/AP_SchoolMaster_DistrictMandal_Mapping.xlsx')

In [61]:
df_unpivot = pd.merge(df_unpivot,school_mapping,how='left',left_on='Udise',right_on='SCHOOLCODE')

In [62]:
df_unpivot = df_unpivot.assign(State='Andhra Pradesh',District='',Topic='',Subtopic='',Skill='',MUA='')

In [63]:
df_unpivot = df_unpivot.rename(columns={'SCHOOLNAME':'School','schooltype':'School Type','Udise':'UDISE'\
                                       ,'Social_Category':'Social Category'})

In [64]:
df_unpivot = df_unpivot.drop(['SCHOOLCODE'],axis=1)

In [66]:
df_unpivot = df_unpivot.assign(District='')

In [67]:
df_unpivot = df_unpivot.rename(columns={'Social_Category':'Social Category'})

In [68]:
df_unpivot = df_unpivot[[
'State',
'District',
'Mandal',
'School',
'UDISE',
'School Type',
'Grade',
'Child Name',
'Child Id',
'Gender',
'Social Category',
'Subject',
'Question no',
'Question Score',
'Topic',
'Subtopic',
'Skill',
'MUA',]]

In [70]:
df_unpivot.to_csv('./CBALongFiles/CBA_G3_LongFile.csv')

In [71]:
df_unpivot.head()

,State,District,Mandal,School,UDISE,School Type,Grade,Child Name,Child Id,Gender,Social Category,Subject,Question no,Question Score,Topic,Subtopic,Skill,MUA
0,Andhra Pradesh,,PEDDAMANDYAM,MPPS PAPE PALLE,28230100101,STATE GOVT,3,VADLA MADIYAN,2013055023,B,4,EVS,1,1,,,,
1,Andhra Pradesh,,PEDDAMANDYAM,MPPS PAPE PALLE,28230100101,STATE GOVT,3,VEPALLI VEMPALLI MUBARAK,2040110072,B,3,EVS,1,1,,,,
2,Andhra Pradesh,,PEDDAMANDYAM,MPPS PAPE PALLE,28230100101,STATE GOVT,3,KANDUKURI RANJITHKUMAR,2043102036,B,3,EVS,1,1,,,,
3,Andhra Pradesh,,PEDDAMANDYAM,MPPS PAPE PALLE,28230100101,STATE GOVT,3,KANDUKURI KandukuriBharathi,2043102048,G,3,EVS,1,1,,,,
4,Andhra Pradesh,,PEDDAMANDYAM,MPPS PAPE PALLE,28230100101,STATE GOVT,3,GANDLAPENTA THANVIR,2043102052,B,3,EVS,1,1,,,,


In [72]:
df_unpivot.shape

(20268516, 18)